In [72]:
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict
from bisect import bisect_left
import pandas as pd
import math
import numpy as np
import csv
%matplotlib inline

In [ ]:
# Importamos los datos de un archivo de texto en forma de una matriz cuadrada:

datos=np.loadtxt('', delimiter=',', dtype=int)
datos

In [114]:
# importación de los datos de un archivo de texto sin ningún orden:

listas = []
datos=[]

with open('frecuencias_ejemplo1.txt', newline='') as archivo:
    for linea in csv.reader(archivo):
        listas.append(linea)
        
for lista in listas:
    for elemento in lista:
        datos.append(float(elemento))
        np_datos=np.sort(np.array(datos))
                         
print('los datos, agrupados, son los siguientes '+ str(np.sort(np_datos)), end=' ')

los datos, agrupados, son los siguientes [ 90.  95.  96. 108. 114. 116. 119. 123. 130. 141. 147. 149. 150. 151.
 154. 157. 163. 171. 172. 175. 178. 183. 185. 202. 206.] 

In [118]:
""" Cálculo de las medidas de tendencia central para los daton no agrupados """

# media 
print('la media es igual a: '+ str(np.mean(np_datos)))

# mediana
print('la media es igual a: '+ str(np.median(np_datos)))

# moda
print('la media es igual a: '+ str(stats.mode(np_datos)))

la media es igual a: 147.0
la media es igual a: 150.0
la media es igual a: ModeResult(mode=array([90.]), count=array([1]))


In [ ]:
"""Cálculo de las cantidades índices de la tabla de frecuencia"""

# El número total de datos es:
np_datos=np.array(datos)
print('El número total de datos(n) es: '+str(len(np_datos)))

# Los máximos y mínimos son:
print('El dato mayor es: '+np_datos.max().astype(str))
print('El dato menor es: '+np_datos.min().astype(str))

# Rango
rango=np_datos.max().astype(float)-np_datos.min().astype(float)
print('El rango(R) es igual a: '+ str(rango))

In [123]:
# Amplitud y número de intervalos de clase utilizando el operador módulo para la verificación de cantidades pares o impares. 
""""""

# Omitase esta función si el número de intervalos de clase es proporcionado. Calculese la amplitud(A) de manera directa mediante (R/k) y coloquese, 
# como parámetro "rests", en el siguiente bloque de código.

# Esta fucnión calcula el número de intervalo de clases:

def inter_clases(k):
    print('Aplicando Sturges [1 + 3.322 (log(n))]: \n'
          '       (\u039A) es igual a ' +'"'+str(round(k,2))+'"'+'. El número aproximado de intervalos sería de ' +str(round(k))+',')
    
    # Confirmación de números pares mediante el módulo de un número:
    if round(k)%2==0:
        print('       pero '+str(round(k))+ ' es un número par. Por convención necesitamos tomar la cifra siguiete, que sea mayor e impar, o sea (\u039A)='+str(round(k+1))+'.\n'
         '       Y la amplitud(A) igual a (R/(\u039A)) es '+str(round(rango/round(k+1))))
        return round(rango/round(k+1))
       
    else:
        print('       y '+str(round(k)) + ' es un numero impar. Por convención tomamos este número como cantidad de intervalos. Entonces (\u039A) es igual a '+str(round(k))+'.\n'
              '       Y la amplitud(A) igual a (R/(\u039A)) es '+str(round(rango/(round(k)))))
        return round(rango/(round(k)))

# Ejecución de la función "inter_clases" para obtener el número de clases (k) y la amplitud (A) en una variable llamada "rests". 
rests=inter_clases(1+(3.322*(math.log10(len(np_datos)))))

El número total de datos(n) es: 25
El dato mayor es: 206.0
El dato menor es: 90.0
El rango(R) es igual a: 116.0
Aplicando Sturges [1 + 3.322 (log(n))]: 
       (Κ) es igual a "5.64". El número aproximado de intervalos sería de 6,
       pero 6 es un número par. Por convención necesitamos tomar la cifra siguiete, que sea mayor e impar, o sea (Κ)=7.
       Y la amplitud(A) igual a (R/(Κ)) es 17.0


In [121]:
# creación de los intervalos:
def intervalo(start, stop, step):
    i=start
    while i <= stop:
        yield i
        i+=step
        yield i

l_inter=[]
for i in intervalo(np_datos.min(), np_datos.max(),rests):
    l_inter.append(i)

# Tupla y después listas con los intervalos en la variable "t_iter":
t_iter=np.array([i for i in zip(l_inter[::2],l_inter[1::2])]).tolist()
t_iter

[[90.0, 107.0],
 [107.0, 124.0],
 [124.0, 141.0],
 [141.0, 158.0],
 [158.0, 175.0],
 [175.0, 192.0],
 [192.0, 209.0]]

In [95]:
# Cálculo de las marcas de clase "x"
""""""

# Las marcas de clase son igual al promedio, entre cada uno d los L.I. y L.S., para cada intervalo de clase. Es necesario la construcción de un arreglo 
# sobre la lista "t_iter" y la aplicación de la función "np.mean()" para calcular el promedio de cada uno de los intervalos:
np_titer=np.array(t_iter)
macl=[np.mean(x) for x in np_titer]

# Cálculo de las frecuecias absolutas
""""""

# Función que realiza el conteo de los datos y asigna para cada intervalo de clase su frecuencia absoluta:
def frecuencia(li, min, max):
    ctr = 0
    for x in li:
        if min <= x < max:
            ctr += 1
    return ctr

# Ejecución de la función "frecuencias" para que pueda iterar sobre los rangos de la lista "t_iter". Los resultados son alojados dentro de la variable 
# "frec_abs"
frec_abs=[]
for d in t_iter:
    frec_abs.append(frecuencia(datos, d[0], d[1]))
    

# Cálulo de la frecuencia relativa
""""""

# Construimos un arreglo sobre la lista "frec_abs" y a cada elemento lo dividimos entre el número total de datos:
np_frec=np.array(frec_abs)
frec_re=[(f/(len(np_datos))) for f in np_frec]

# Cálculo de la frecuencia absoluta acumulada
""""""

# Pasamos como parámetro, a la función "np.cumsum", el arreglo "frec_abs":
frec_abs_ac=list(np.cumsum(frec_abs))



In [122]:
df_frec=pd.DataFrame({'clases':t_iter,'marcas_clase':macl, 'frec_absoluta':frec_abs, 'frec_relativa':frec_re, 'frec_abs_acumulada':frec_abs_ac})
df_frec

,clases,marcas_clase,frec_absoluta,frec_relativa,frec_abs_acumulada
0,"[90.0, 107.0]",98.5,3,0.12,3
1,"[107.0, 124.0]",115.5,5,0.20,8
2,"[124.0, 141.0]",132.5,1,0.04,9
3,"[141.0, 158.0]",149.5,7,0.28,16
4,"[158.0, 175.0]",166.5,3,0.12,19
5,"[175.0, 192.0]",183.5,4,0.16,23
6,"[192.0, 209.0]",200.5,2,0.08,25
